This Notebook will be mainly used for Capstone project of Coursera

In [7]:
import pandas as pd
import numpy as np

In [17]:
#Assigning url to a variable
url1 = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Fetching table by using pandas read_html method
table1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# We only need part of the fetched object
postal_code = table1[0]

#removing not assigned boroughs
for item in range(len(postal_code['Borough'])):
    if postal_code.loc[item]['Borough'] == 'Not assigned':
        postal_code.drop(item,inplace=True)

# removing / and putting ,
for item in range(len(postal_code['Neighborhood'])):
    postal_code.iloc[item]['Neighborhood']=postal_code.iloc[item]['Neighborhood'].replace('/',',')

postal_code.shape

(103, 3)

In [18]:
#since geocoder did not work, we need to read coordinates from csv file
locations = pd.read_csv('Geospatial_Coordinates.csv')
latitude =[]
longitude=[]

#Adding latitude and longitude columns to main dataframe
for item in postal_code['Postal code']:
    for i, code in enumerate(locations['Postal Code']):
        if item == code:
            latitude.append(locations.iloc[i][1])
            longitude.append(locations.iloc[i][2])
postal_code['Latitude']=latitude
postal_code['Lonitude']=longitude
postal_code.head()

,Postal code,Borough,Neighborhood,Latitude,Lonitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [12]:
import folium

#geolocation of Toronto for initializing the map
latitude_tor = 43.651070
longitude_tor = -79.347015

#creating map
map_toronto = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=11)

#adding markers to the map
for lat, lng, borough, neighborhood in zip(postal_code['Latitude'], postal_code['Lonitude'], postal_code['Borough'], postal_code['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto

Let's see how close what we have from clustering and real borough data. In order to see the difference, we first create 10 clusters with sklearn and then implement on the map, then we use real borough data to implement locations on the map. 

In [20]:
#clustering by Boroughs

from sklearn.cluster import KMeans

kclusters = 10

toronto_cluster = postal_code.drop(['Neighborhood','Borough','Postal code'],1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_cluster)

In [21]:
#Checking Cluster labels
kmeans.labels_[:]

array([1, 1, 5, 0, 5, 9, 3, 4, 1, 5, 8, 9, 3, 1, 1, 5, 8, 9, 3, 1, 5, 2,
       3, 1, 5, 8, 7, 4, 0, 1, 5, 2, 7, 4, 0, 1, 5, 5, 7, 4, 0, 1, 5, 5,
       1, 4, 0, 1, 5, 0, 6, 7, 4, 0, 1, 8, 2, 6, 1, 4, 0, 8, 8, 2, 6, 7,
       4, 8, 8, 2, 6, 7, 4, 8, 5, 2, 9, 6, 7, 8, 5, 2, 7, 8, 5, 7, 8, 5,
       9, 6, 7, 5, 5, 9, 6, 3, 5, 5, 2, 5, 1, 2, 9])

In [22]:
#adding cluster labels to geopositional dataframe
toronto_cluster['Cluster Label'] = kmeans.labels_

In [26]:
toronto_cluster['Cluster Label'].unique()

array([1, 5, 0, 9, 3, 4, 8, 2, 7, 6])

In [29]:
#drawing map with clusters in different colors

#geolocation of Toronto for initializing the map
latitude_tor = 43.651070
longitude_tor = -79.347015

#creating map
map_toronto_cluster = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=10)

#adding markers to the map complying with clusters
cluster_colors=['red','green','blue','cyan','black','yellow','pink','purple','brown','orange']
for lat, lng, cluster in zip(toronto_cluster['Latitude'], toronto_cluster['Lonitude'], toronto_cluster['Cluster Label']):
    label = 'This neighborhood belongs to cluster no {}'.format(cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=cluster_colors[cluster],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_cluster) 
map_toronto_cluster

In [39]:
#using real borough data to cluster neighborhoods

#creating map
map_toronto_boroughs = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=10)

#adding markers to the map complying with real boroughs

#making a dictionary for coloring boroughs
borough_colors=['red','green','blue','cyan','black','yellow','pink','purple','brown','orange']
borough_names=postal_code['Borough'].unique()
borough_coloring= dict(zip(borough_names,borough_colors))
#
for lat, lng, borough in zip(postal_code['Latitude'], postal_code['Lonitude'], postal_code['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_coloring[borough],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_boroughs) 
map_toronto_boroughs

We can clearly see that real borough positions and cluster positions are not exactly the same.